In [136]:
class UnstructuredDb():
    _instance = None

 
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(UnstructuredDb, cls).__new__(cls, *args, **kwargs)
        return cls._instance
    
    def __init__(self,path:str="embedded-database/",table:str='ingredients'):
        import chromadb
        import os
        from chromadb.db.base import UniqueConstraintError
        from chromadb.utils import embedding_functions
        
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            print(f"Directory '{path}' created successfully.")

         
        
        client = chromadb.PersistentClient(path=path)
        em = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="Huffon/sentence-klue-roberta-base")
        try:
            self.collection = client.create_collection(name=table, embedding_function=em)
        except UniqueConstraintError: 
            self.collection = client.get_collection(name=table, embedding_function=em)
            
    def get(self,text:str,n:int=2)->[str]:
        

        results=self.collection.query(query_texts=[text],n_results=n)
        urls=[id for id in results['ids'][0]]

        return urls
        
  

    def add(self,key:str=None,ingredient:str=None,metadata:dict=None):
        
        self.collection.add(
        documents=[ingredient],
        metadatas=[metadata],
        ids=[key]
        )


In [137]:
import pickle

class RecipeDict:
    _instance = None
    _is_loaded = False

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(RecipeDict, cls).__new__(cls)
        return cls._instance

    def load_data(self, file_path):
        if not self._is_loaded:
            with open(file_path, 'rb') as file:
                self.data = pickle.load(file)
            self._is_loaded = True
        return self.data

In [138]:
unstructured_db=UnstructuredDb()
singleton_loader = RecipeDict()
recipe_dict = singleton_loader.load_data('recipes.pkl')

In [139]:
from flask import Flask, jsonify,send_from_directory,request,session,make_response
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from sqlalchemy import func
import os
from dotenv import load_dotenv

In [140]:

load_dotenv()
dist_path=os.getenv("DIST_PATH")

In [141]:
app = Flask(__name__, static_folder=dist_path, static_url_path='')
app.config["SECRET_KEY"] = "apple"
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)

In [142]:
@app.route('/')
def index():
    return send_from_directory(dist_path, 'index.html')

In [143]:
@app.route('/assets/<path:path>')
def serve_assets(path):
    os_name = os.name
    if os_name == 'nt': # 서버가 윈도우 일경우
        print("Current OS: Windows")
        if path.endswith('.js'):
            response = make_response(send_from_directory(os.path.join(dist_path, 'assets'), path))
            response.headers.set('Content-Type', 'application/javascript')
        elif path.endswith('.css'):
            response = make_response(send_from_directory(os.path.join(dist_path, 'assets'), path))
            response.headers.set('Content-Type', 'text/css')
        else:
            response = send_from_directory(os.path.join(dist_path, 'assets'), path)
        return response
    
    return send_from_directory(os.path.join(dist_path, 'assets'), path)

In [144]:
class User(db.Model):
    id = db.Column(db.String, primary_key=True)
    nickname = db.Column(db.String, primary_key=True, unique=True, nullable=False)
    password = db.Column(db.String, nullable=False)

In [145]:
class Refrigerator(db.Model):
    id = db.Column(db.Integer, primary_key=True, autoincrement=True, nullable=False)
    nickname = db.Column(db.String, nullable=False)
    ingredient = db.Column(db.String, nullable=False)
    store_method = db.Column(db.Boolean)
    count = db.Column(db.Integer)
    expire_date = db.Column(db.Date)


In [146]:
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    contents = db.Column(db.String(50), nullable=False)
    nickname = db.Column(db.String(50), nullable=False)
    image = db.Column(db.String(200), nullable=True)
    created_at = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            'id': self.id,
            'title': self.title,
            'contents': self.contents,
            'nickname': self.nickname,
            'image': self.image,
            'created_at': self.created_at.isoformat()
        }


In [147]:
@app.route('/signup', methods=['POST'])
def signup():
    data = request.get_json()
    id = data['id']
    nickname = data['nickname']
    password = data['password']
    
    # ID와 닉네임의 사용 가능 여부를 확인합니다.
    available_id = User.query.filter_by(id=id).first()
    available_nickname = User.query.filter_by(nickname=nickname).first()
    
    if available_id:
        return "이미 존재하는 아이디 입니다!",400
    elif available_nickname:
        return "이미 존재하는 이름 입니다!",400
    
    # 새로운 사용자 생성 및 데이터베이스에 추가
    new_user = User(id=id, nickname=nickname, password=password)
    db.session.add(new_user)
    db.session.commit()
    session['user']=id
    session.modified = True
    return "회원가입 성공!",200




In [148]:
@app.route('/login',methods=['POST'] )  
def login():
    data = request.get_json()
    id = data['id']
    password = data['password']
    user_info = User.query.filter_by(id=id).filter_by(password=password).first()
    if user_info:
        session['user']=user_info.id
        print(session['user'])
        print(session.items())
        return "로그인 성공!",200
    return "회원정보가 일치하지 않습니다!",200

In [149]:
@app.route('/myinfo',methods=['GET'] )  
def myinfo():
    print(session['user'])
    if 'user' in session:
        
        return session['user'], 200
    return "로그인 해주세요!",200

In [150]:
@app.route('/logout',methods=['GET'] )  
def logout():
    if 'user' in session:
        session.pop('user', None)
        session.modified = True
        return "로그아웃 되었습니다!",200
    return "로그인 해주세요!",400

In [151]:
@app.route('/add',methods=['POST'] )  
def add():
    datas = request.get_json()
    print(datas)
    if 'user' in session:

        
        for data in datas:
            ingredient = data['ingredient']
            count =data['ingredientCount']
            store_method = data['storeMethod']
            expire_date = data['expireDate']
            expire_date = datetime.strptime(expire_date, "%Y-%m-%d").date()
            new_ingredient = Refrigerator(
                ingredient=ingredient,
                count=count,
                store_method=store_method,
                expire_date=expire_date,
                nickname=session['user']
            )
            db.session.add(new_ingredient)
        db.session.commit()

    
        
        return "성공!",200
    return "로그인해주세요!",400

In [152]:
@app.route('/get', methods=['GET'])
def get():
    print(session['user'])
    if 'user' in session:
        ingredients = Refrigerator.query.filter_by(nickname=session['user']).all()
        
        # 데이터를 JSON 형식으로 변환하기 위해 각 항목을 사전(dict) 형태로 변환
        ingredients_list = []
        for ingredient in ingredients:
            ingredient_dict = {
                'ingredient':ingredient.ingredient,
                'count':ingredient.count,
                'storeMethod':ingredient.store_method,
                'expireDate':ingredient.expire_date,
                'id':ingredient.id
            }
            ingredients_list.append(ingredient_dict)
            
        objs = db.session.query(Refrigerator).filter_by(id=session['user']).all()
        ingredients = ', '.join([obj.ingredient for obj in objs])
        print(ingredients)
        # JSON 형식으로 클라이언트에게 응답
        return jsonify(ingredients_list), 200
    
    # 로그인하지 않은 경우 메시지 반환
    return jsonify({'message': '로그인해주세요!'}), 200

In [153]:
@app.route('/delete', methods=['GET'])
def delete():
    if 'user' in session:
        id = request.args.get('id')  # GET 요청의 경우 request.args 사용
        obj = db.session.query(Refrigerator).filter_by(id=id).first()
        if obj:
            db.session.delete(obj)
            db.session.commit()
            return "success"
        else:
            return "failed"

    return "failed"


In [154]:
@app.route('/modify', methods=['POST'])
def modify():
    if 'user' in session:
        datas = request.get_json()
        objs = db.session.query(Refrigerator).filter_by(nickname=session['user']).all()
        for obj in objs:
            db.session.delete(obj)
        for data in datas:
            expire_date_str = data['expire_date']
            if 'GMT' in expire_date_str:
                expire_date = datetime.strptime(expire_date_str, '%a, %d %b %Y %H:%M:%S GMT').date()
            else:
                expire_date = datetime.strptime(expire_date_str, '%Y-%m-%d').date()
            
            new_data = Refrigerator(
                nickname=session['user'],
                ingredient=data['ingredient'],
                store_method=data['store_method'],
                count=data['count'],
                expire_date=expire_date
            )
            db.session.add(new_data)
        db.session.commit()
        
        return "success"
    return "failed"


In [155]:
@app.route('/search/<int:n>', methods=['GET'])  

def search(n):
    if 'user' in session:
        objs = db.session.query(Refrigerator).filter_by(nickname=session['user']).all()
        ingredients = ', '.join([obj.ingredient for obj in objs])
        print(ingredients)
        recipes=unstructured_db.get(text=ingredients,n=n)
        results=[]
        for recipe_id in recipes:
            results.append(recipe_dict[recipe_id])
        
        return jsonify(results), 200
    return "failed",200

In [156]:
@app.route('/post/<int:post_id>', methods=['GET'])
def get_post(post_id):
    post = Post.query.get(post_id)
    is_author=False
    if 'user' in session and session['user'] == post.nickname:
        is_author=True
        
    return jsonify({
        'id': post.id,
        'title': post.title,
        'contents': post.contents,
        'image': post.image,
        'created_at': post.created_at,
        'isAuthor': is_author
    })

In [157]:
@app.route('/post/modify/<int:post_id>', methods=['POST'])
def update_post(post_id):
    data = request.get_json()
    post = Post.query.get(post_id)
    if not post:
        return jsonify({'error': 'Post not found'}), 404


    post.title = data['title']
    post.contents = data['contents']
    
    db.session.commit()
    return jsonify({'message': 'Post updated successfully'})

In [158]:
@app.route('/post/add', methods=['POST'])
def add_post():
    data = request.get_json()
    title = data['title']
    contents = data['contents']
    nickname = session['user']  # 로그인된 사용자의 닉네임
    print('sssssssssssssssssssssss')
    print(nickname,contents,title)
        

    new_post = Post(title=title, contents=contents, nickname=nickname)
    db.session.add(new_post)
    db.session.commit()
    return jsonify({'message': 'Post created successfully'})

In [159]:
@app.route('/post/list/<int:pageNumber>', methods=['GET'])
def get_post_list(pageNumber):
    
    items_per_page = 10
    offset = (pageNumber - 1) * items_per_page

    total_posts = db.session.query(func.count(Post.id)).scalar()
    total_pages = (total_posts + items_per_page - 1) // items_per_page

    paginated_posts = Post.query.order_by(Post.created_at.desc()).offset(offset).limit(items_per_page).all()
    
    return jsonify({
        'data': [post.to_dict() for post in paginated_posts],
        'totalPages': total_pages
    })



In [160]:
@app.route('/post/delete/<int:post_id>')
def delete_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        return jsonify({'error': 'Post not found'}), 404


    db.session.delete(post)

    db.session.commit()
    return jsonify({'message': 'Post updated successfully'})

In [161]:
if __name__ == '__main__':
    with app.app_context():
        db.create_all() 
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.219.105:5000
Press CTRL+C to quit
C:\Users\Desktop\AppData\Local\Temp\ipykernel_14384\3969683745.py:3: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  post = Post.query.get(post_id)
192.168.219.105 - - [07/Jun/2024 19:51:40] "GET /post/3 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:51:41] "GET /myinfo HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:51:41] "GET /post/3 HTTP/1.1" 200 -


s


C:\Users\Desktop\AppData\Local\Temp\ipykernel_14384\1512690668.py:4: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  post = Post.query.get(post_id)
192.168.219.105 - - [07/Jun/2024 19:51:46] "POST /post/modify/3 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET / HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /assets/index-BDSB2b-I.js HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /assets/index-Ctg9QYGi.css HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /myinfo HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /images/recipe.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /images/fridge.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:52:43] "GET /images/community.

Current OS: Windows
Current OS: Windows
s


...

Current OS: Windows
Current OS: Windows


GET /images/fridge.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:23] "GET /images/recipe.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:23] "GET /images/community.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:23] "GET /myinfo HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:23] "GET /vite.svg HTTP/1.1" 304 -


s


192.168.219.105 - - [07/Jun/2024 19:55:23] "GET /post/list/1 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:25] "GET /post/4 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:26] "GET /post/delete/4 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:26] "GET /post/list/1 HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:37] "GET /myinfo HTTP/1.1" 200 -
192.168.219.105 - - [07/Jun/2024 19:55:37] "GET /images/logo.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:37] "GET /images/fridge.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:37] "GET /images/recipe.png HTTP/1.1" 304 -
192.168.219.105 - - [07/Jun/2024 19:55:37] "GET /images/community.png HTTP/1.1" 304 -


s


In [162]:
ls/myproject/server/dist/assets/images

스위치가 틀립니다 - "myproject".
